In [168]:
import torch.nn.functional as F
from torch.autograd import Variable
import copy
from collections import Counter
import torch 
import numpy as np
import torch.nn as nn
import torchvision

In [86]:
def read_data(path):
    f = open(path, "r")
    lines = f.readlines()
    f.close()
    data = []
    for l in lines:
        labelSplit = l.replace('\n','').split(' ', 1)
        data.append([labelSplit[0], [word.lower() for word in labelSplit[1].split()]])
    return data

data = read_data('./questions.txt')

In [87]:
def remove_stop_words(data, path):
    stop_words = []
    with open(path) as f:
        stop_words = [word for line in f for word in line.split(",")]
    data_without_stop_words = []
    for k, v in data:
        words = [t for t in v if t not in stop_words]
        data_without_stop_words.append((k, words))
    return data_without_stop_words

data = remove_stop_words(data, './stop_words.txt')

In [88]:
def get_labels(data):
    _labels = []
    for k,v in data:
        _labels.append(k)   
    _unique_label = list(set(_labels))
    _unique_label_dict = {}
    for k,v in enumerate(_unique_label):
        _unique_label_dict[v] = k
    return _unique_label_dict

labels = get_labels(data)

In [89]:
def append_labels(data, labels):        
    cleaned_data = []
    for k,v in data:
        cleaned_data.append((labels[k],v))
        
    return np.array(cleaned_data)

data = append_labels(data, labels)

In [90]:
def create_indexed_vocab(data):
    vocab = []
    for _, sent in data:
        for word in sent:
            vocab.append(word)
    count = Counter(vocab)
    count = {w : count[w] for w in count if count[w] >= 2}
    vocab = []
    for k, v in count.items():
        vocab.append(k)
    indexed_vocab = {word: idx for idx, word in enumerate(vocab)}
    return indexed_vocab
def create_vocab(data):
    total_words_orig = []
    for k,sent in data:
        for word in sent:
            total_words_orig.append(word)
    total_words = list(set(total_words_orig))
    total_words_str = ' '.join(total_words)
    vocab = set(total_words_str.split()) 
    word2idx = {word: idx for idx, word in enumerate(vocab)} # create word index
    return word2idx
word2idx=create_vocab(data)
word2idx['#UNK#'] = len(word2idx)
indexed_vocab = create_indexed_vocab(data)
indexed_vocab['#UNK#'] = len(indexed_vocab)

In [93]:
def load_glove_embeddings(path, indexed_vocab, embedding_dim=300):
    with open(path) as f:
        embeddings = np.zeros((len(indexed_vocab), embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = indexed_vocab.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

glove_random = load_glove_embeddings('./glove.small.txt', indexed_vocab)
glove_pre = load_glove_embeddings('./glove.small.txt', word2idx)
embeddings_random = nn.Embedding(glove_random.size(0), glove_random.size(1))
embeddings_pretrained = nn.Embedding.from_pretrained(glove_pre, freeze=True)

In [94]:
def split_train_test(data, test_ratio):
    data_copy = copy.deepcopy(data)
    np.random.shuffle(data_copy)
    test_set_size = int(len(data) * test_ratio)
    test = data_copy[:test_set_size]
    train = data_copy[test_set_size:]
    return train, test

train, test = split_train_test(data, 0.2)
train,dev =split_train_test(train,0.1)

In [95]:
class BOWClassifier(nn.Module):
    def __init__(self,input_size,hidden_size, num_labels):
        super(BOWClassifier, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.input_size, num_labels)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(x)
        #output = self.sigmoid(output)
        return output

In [101]:
def make_bow_vector(sentence, indexed_vocab,embeddings):
    pt_tensor= torch.zeros(300, dtype=torch.long)
    count = 0
    for word in sentence:
        count += 1
        if word in indexed_vocab:
            pt_tensor = torch.add(pt_tensor, embeddings(torch.LongTensor([indexed_vocab[word]]))[0])
        else:
            pt_tensor = torch.add(pt_tensor, embeddings(torch.LongTensor([indexed_vocab['#UNK#']]))[0])
    pt_tensor=torch.div(pt_tensor, count)
    return pt_tensor

def get_bow_rep(data,word2idx,embeddings):
    bow_data = []
    for label, sent in data:
        bow_data.append(make_bow_vector(sent, word2idx,embeddings))
    return torch.stack(bow_data)
        
training_set = get_bow_rep(train,word2idx,embeddings_pretrained)
training_set_rand = get_bow_rep(train,indexed_vocab,embeddings_random)

In [102]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 300
hidden_size = 100
num_classes = 51
num_epochs = 350
learning_rate = 0.0001
batch_size = 100

model = BOWClassifier(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
#training in batches
for epoch in range(num_epochs):
    permutation = torch.randperm(training_set.size()[0])
    for i in range(0, training_set.size()[0], batch_size):
        optimizer.zero_grad()
        indices = permutation[i:i + batch_size]
        batch_features = training_set[indices]
        batch_features = batch_features.reshape(-1, 300).to(device)
        batch_labels = torch.LongTensor([label for label, sent in train[indices]]).to(device)
        batch_outputs = model(batch_features)
        loss = criterion(batch_outputs, batch_labels)
        loss.backward()
        optimizer.step()
    print(loss.item())
    print(epoch)

3.8577375411987305
0
3.766239881515503
1
3.7015557289123535
2
3.6341633796691895
3
3.5899248123168945
4
3.461988687515259
5
3.5528314113616943
6
3.463732957839966
7
3.2996716499328613
8
3.40735125541687
9
3.2944467067718506
10
3.2035000324249268
11
3.2594258785247803
12
3.093846321105957
13
3.092564105987549
14
3.131394863128662
15
3.1297993659973145
16
2.9859044551849365
17
3.0528383255004883
18
2.8519093990325928
19
2.9279050827026367
20
2.905634641647339
21
2.8870065212249756
22
2.7975986003875732
23
2.788588523864746
24
2.7590572834014893
25
2.6858606338500977
26
2.706007719039917
27
2.9241583347320557
28
2.818058729171753
29
2.7365477085113525
30
2.747945547103882
31
2.7600347995758057
32
2.8307321071624756
33
2.7383768558502197
34
2.7723028659820557
35
2.5529563426971436
36
2.568894386291504
37
2.33998703956604
38
2.6940736770629883
39
2.386039972305298
40
2.563384532928467
41
2.7597479820251465
42
2.4870269298553467
43
2.408236503601074
44
2.4940176010131836
45
2.575275897979736

In [103]:
#testing
test_len =len(test)
correct=0
for label,data in test:
    bow_vec = make_bow_vector(data, word2idx,embeddings_pretrained)
    logprobs = model(bow_vec)
    logprobs = F.softmax(logprobs)
    pred = np.argmax(logprobs.data.numpy())
    if pred==label:
        correct+=1
    print('prediction: {}'.format(pred))
    print('actual: {}'.format(label))
accuracy = correct/test_len
print('accuracy: {}'.format(accuracy))

C:\Users\joshy\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


prediction: 17
actual: 17
prediction: 39
actual: 1
prediction: 5
actual: 5
prediction: 24
actual: 24
prediction: 12
actual: 12
prediction: 13
actual: 13
prediction: 12
actual: 12
prediction: 1
actual: 1
prediction: 11
actual: 11
prediction: 39
actual: 1
prediction: 4
actual: 4
prediction: 42
actual: 42
prediction: 24
actual: 24
prediction: 32
actual: 44
prediction: 13
actual: 13
prediction: 17
actual: 17
prediction: 37
actual: 37
prediction: 17
actual: 17
prediction: 2
actual: 2
prediction: 32
actual: 32
prediction: 13
actual: 13
prediction: 4
actual: 4
prediction: 17
actual: 24
prediction: 20
actual: 20
prediction: 20
actual: 20
prediction: 12
actual: 1
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 24
actual: 24
prediction: 10
actual: 10
prediction: 13
actual: 1
prediction: 13
actual: 48
prediction: 11
actual: 11
prediction: 17
actual: 17
prediction: 32
actual: 10
prediction: 45
actual: 45
prediction: 13
actual: 13


actual: 32
prediction: 12
actual: 37
prediction: 2
actual: 2
prediction: 11
actual: 11
prediction: 11
actual: 11
prediction: 37
actual: 37
prediction: 17
actual: 17
prediction: 42
actual: 42
prediction: 17
actual: 17
prediction: 17
actual: 10
prediction: 32
actual: 32
prediction: 20
actual: 20
prediction: 13
actual: 1
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 39
actual: 39
prediction: 39
actual: 39
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 11
actual: 11
prediction: 13
actual: 12
prediction: 2
actual: 2
prediction: 17
actual: 17
prediction: 48
actual: 41
prediction: 37
actual: 49
prediction: 12
actual: 12
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 30
actual: 50
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 46
actual: 46
prediction: 17
actual: 17
prediction: 12
actual: 17
prediction: 17
actual: 37
prediction: 39
actual: 39
prediction: 12
actual: 12
prediction: 12
actual: 12
prediction: 28
actual: 28
predic

prediction: 17
actual: 49
prediction: 17
actual: 17
prediction: 48
actual: 48
prediction: 11
actual: 48
prediction: 42
actual: 37
prediction: 46
actual: 46
prediction: 12
actual: 12
prediction: 17
actual: 17
prediction: 13
actual: 43
prediction: 12
actual: 12
prediction: 17
actual: 48
prediction: 17
actual: 17
prediction: 28
actual: 13
prediction: 32
actual: 5
prediction: 17
actual: 17
prediction: 12
actual: 12
prediction: 11
actual: 11
prediction: 42
actual: 17
prediction: 12
actual: 12
prediction: 50
actual: 50
prediction: 11
actual: 11
prediction: 4
actual: 4
prediction: 10
actual: 48
prediction: 11
actual: 11
prediction: 12
actual: 12
prediction: 47
actual: 48
prediction: 42
actual: 42
prediction: 46
actual: 46
prediction: 32
actual: 32
prediction: 13
actual: 13
prediction: 17
actual: 17
prediction: 11
actual: 18
prediction: 17
actual: 17
prediction: 11
actual: 34
prediction: 12
actual: 12
prediction: 17
actual: 17
prediction: 13
actual: 13
prediction: 10
actual: 10
prediction: 10


prediction: 17
actual: 17
prediction: 11
actual: 17
prediction: 13
actual: 13
prediction: 1
actual: 1
prediction: 13
actual: 48
prediction: 17
actual: 17
prediction: 2
actual: 42
prediction: 17
actual: 42
prediction: 12
actual: 39
prediction: 2
actual: 42
prediction: 17
actual: 17
prediction: 39
actual: 48
prediction: 45
actual: 50
prediction: 13
actual: 43
prediction: 36
actual: 36
prediction: 42
actual: 42
prediction: 12
actual: 12
prediction: 11
actual: 11
prediction: 13
actual: 13
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 11
actual: 11
prediction: 13
actual: 13
prediction: 10
actual: 10
prediction: 17
actual: 42
prediction: 2
actual: 2
prediction: 46
actual: 46
prediction: 13
actual: 13
prediction: 45
actual: 45
prediction: 17
actual: 17
prediction: 20
actual: 20
prediction: 1
actual: 10
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 1
actua

In [208]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        # Decode the hidden state of the last time step
        tag_space = self.hidden2tag(lstm_out[-1, :])
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [209]:
#LSTM training
vocab_size=glove_random.size(0)
embedding_dim=glove_random.size(1)
tagset_size=51
hidden_dim=300
learning_rate=0.0001
ModelLSTM = LSTMTagger(embedding_dim,hidden_dim,vocab_size,tagset_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ModelLSTM.parameters(), lr=learning_rate)
num_epochs=2
for epoch in range(num_epochs):
    for label,sent in train:
        ids=[]
        unk = 3375
        for word in sent:
            index =indexed_vocab.get(word)
            if index:
                ids.append([indexed_vocab[word]])
            else:
                ids.append([unk])
        batch_labels=torch.tensor([label])
        sentence=torch.tensor(ids).squeeze()
        optimizer.zero_grad()
        batch_outputs = ModelLSTM(sentence)
        loss = criterion(batch_outputs, batch_labels)
        loss.backward()
        optimizer.step()
        print("loss",loss.item())
    print("epoch",epoch)

loss 4.0579915046691895
loss 3.952364206314087
loss 3.8755927085876465
loss 3.9699630737304688
loss 3.9575557708740234
loss 4.022512912750244
loss 3.9767160415649414
loss 3.9418485164642334
loss 3.9164927005767822
loss 3.998412609100342
loss 3.7816357612609863
loss 3.96309494972229
loss 3.7041828632354736
loss 3.950676441192627
loss 3.9202544689178467
loss 3.7196459770202637
loss 3.6910598278045654
loss 4.054460048675537
loss 3.9719340801239014
loss 3.6004514694213867
loss 3.880077838897705
loss 3.8788983821868896
loss 3.769232749938965
loss 4.032820701599121
loss 3.779816150665283
loss 3.893718957901001
loss 3.525027275085449
loss 3.5001769065856934
loss 3.485808849334717
loss 3.4839584827423096
loss 3.8107099533081055
loss 4.174908638000488
loss 3.8704936504364014
loss 3.51806378364563
loss 3.5130391120910645
loss 3.7875373363494873
loss 3.9925966262817383
loss 3.9984984397888184
loss 3.2746200561523438
loss 3.639558792114258
loss 3.950057029724121
loss 3.3692259788513184
loss 3.8308

loss 2.163680076599121
loss 1.96592116355896
loss 4.089487075805664
loss 3.63022518157959
loss 5.056349277496338
loss 3.319669485092163
loss 3.7121286392211914
loss 5.068061828613281
loss 3.5686917304992676
loss 3.978055477142334
loss 3.0010318756103516
loss 3.2508764266967773
loss 2.7521910667419434
loss 4.821379661560059
loss 4.472434043884277
loss 3.014894485473633
loss 2.9579854011535645
loss 2.626352548599243
loss 2.1210875511169434
loss 2.9010698795318604
loss 1.8819340467453003
loss 3.099241256713867
loss 3.190354824066162
loss 3.417538642883301
loss 4.042081832885742
loss 3.6702096462249756
loss 6.14886474609375
loss 2.840144157409668
loss 4.080549716949463
loss 2.8039512634277344
loss 4.834531784057617
loss 4.807199954986572
loss 3.6392099857330322
loss 1.770263671875
loss 2.605659246444702
loss 2.5858421325683594
loss 3.944770336151123
loss 3.903928518295288
loss 2.803770065307617
loss 2.5229549407958984
loss 2.7846882343292236
loss 3.5870885848999023
loss 2.4993205070495605


loss 4.432937145233154
loss 3.2803711891174316
loss 1.9075747728347778
loss 2.7061524391174316
loss 3.410834550857544
loss 3.0574355125427246
loss 4.471863746643066
loss 2.3131022453308105
loss 4.799829483032227
loss 2.9308409690856934
loss 2.780827760696411
loss 2.9156582355499268
loss 2.4827651977539062
loss 2.923964023590088
loss 4.485927104949951
loss 2.8791892528533936
loss 2.7785911560058594
loss 4.776829719543457
loss 1.7722314596176147
loss 1.8536403179168701
loss 3.618678569793701
loss 1.9577102661132812
loss 2.3700413703918457
loss 5.125541687011719
loss 2.7792577743530273
loss 1.598524808883667
loss 3.1537303924560547
loss 3.0947303771972656
loss 2.409118890762329
loss 3.915496826171875
loss 4.295114517211914
loss 3.7050833702087402
loss 4.944533824920654
loss 2.332041025161743
loss 1.8654885292053223
loss 1.7605867385864258
loss 3.5412845611572266
loss 2.1063272953033447
loss 1.8100788593292236
loss 4.043005466461182
loss 5.303862571716309
loss 2.379953384399414
loss 2.8156

loss 1.0485111474990845
loss 3.210000991821289
loss 2.443678855895996
loss 3.942934989929199
loss 4.174262523651123
loss 3.117579936981201
loss 1.2578953504562378
loss 2.1885805130004883
loss 2.1414809226989746
loss 2.783107280731201
loss 1.2850321531295776
loss 1.2893505096435547
loss 5.228823184967041
loss 1.3695791959762573
loss 1.2467045783996582
loss 1.7137662172317505
loss 2.825902223587036
loss 3.5448317527770996
loss 1.2500648498535156
loss 1.0867646932601929
loss 3.655433177947998
loss 2.329522132873535
loss 3.4238710403442383
loss 3.692181348800659
loss 3.544340133666992
loss 3.3656513690948486
loss 3.2020018100738525
loss 1.2783284187316895
loss 3.2065677642822266
loss 3.892335891723633
loss 3.6609065532684326
loss 4.935759544372559
loss 1.231245756149292
loss 3.578686475753784
loss 3.0685622692108154
loss 5.520140647888184
loss 4.923688888549805
loss 3.0049118995666504
loss 1.0672229528427124
loss 0.8351563215255737
loss 4.1840434074401855
loss 3.5910587310791016
loss 2.600

loss 2.968086004257202
loss 4.913405418395996
loss 2.2864773273468018
loss 2.049898862838745
loss 3.1169464588165283
loss 3.3832364082336426
loss 3.051168918609619
loss 3.312002658843994
loss 0.8766416907310486
loss 2.3003509044647217
loss 0.5022731423377991
loss 2.12066650390625
loss 2.9121665954589844
loss 1.1527559757232666
loss 2.991163492202759
loss 3.164276361465454
loss 2.0094165802001953
loss 2.575183868408203
loss 2.0077266693115234
loss 2.907907724380493
loss 5.161214351654053
loss 2.0290637016296387
loss 0.46779105067253113
loss 3.065117597579956
loss 4.790889739990234
loss 0.5596030950546265
loss 1.3141791820526123
loss 1.1639708280563354
loss 3.5524775981903076
loss 2.1844091415405273
loss 4.605328559875488
loss 5.3277692794799805
loss 0.998358964920044
loss 1.0451970100402832
loss 2.904367446899414
loss 2.2385568618774414
loss 2.3239524364471436
loss 3.6726160049438477
loss 2.500000476837158
loss 4.726902008056641
loss 1.6743509769439697
loss 1.299971580505371
loss 2.7366

...

loss 3.0073063373565674
loss 0.20055636763572693
loss 0.40158480405807495
loss 0.028157025575637817
loss 3.127542495727539
loss 0.024203885346651077
loss 2.3977136611938477
loss 5.599328517913818
loss 0.5167397856712341
loss 1.3972617387771606
loss 0.012409145012497902
loss 0.06351456046104431
loss 0.047097958624362946
loss 0.6555743217468262
loss 0.04367341846227646
loss 0.033355262130498886
loss 0.5241056680679321
loss 0.20370127260684967
loss 0.10232485085725784
loss 5.1557936668396
loss 2.9018731117248535
loss 0.0363856703042984
loss 3.126014232635498
loss 0.16281521320343018
loss 0.0575975701212883
loss 1.0309193134307861
loss 0.24327325820922852
loss 5.244283676147461
loss 0.07573866099119186
loss 1.2785577774047852
loss 1.2159711122512817
loss 0.0130317909643054
loss 2.1147990226745605
loss 3.1210744380950928
loss 0.4776792824268341
loss 0.06188604235649109
loss 2.823694944381714
loss 2.2191672325134277
loss 1.6279010772705078
loss 0.3605769872665405
loss 1.718719244003296
loss 

loss 0.6667457818984985
loss 3.1946043968200684
loss 0.030611775815486908
loss 0.035801783204078674
loss 4.641374588012695
loss 0.1258750855922699
loss 0.17285794019699097
loss 0.7932654619216919
loss 0.8453944325447083
loss 1.8880970478057861
loss 0.031765181571245193
loss 2.994785785675049
loss 0.024716664105653763
loss 0.3212481737136841
loss 0.12556275725364685
loss 3.7265865802764893
loss 0.2590689957141876
loss 2.2100019454956055
loss 0.08882144093513489
loss 4.148281097412109
loss 0.32464975118637085
loss 2.1910459995269775
loss 1.167040467262268
loss 0.1323428750038147
loss 0.3850560188293457
loss 2.127579689025879
loss 0.02673659473657608
loss 0.0728030800819397
loss 3.149872303009033
loss 2.5858354568481445
loss 4.817178726196289
loss 0.11455748975276947
loss 0.8002841472625732
loss 2.449413776397705
loss 1.5311369895935059
loss 0.4030791223049164
loss 1.4346596002578735
loss 0.4739258885383606
loss 3.0191826820373535
loss 0.6679820418357849
loss 3.1561312675476074
loss 1.589

loss 0.12418980896472931
loss 2.4015228748321533
loss 0.31162774562835693
loss 0.08181500434875488
loss 3.8828835487365723
loss 1.3473352193832397
loss 1.1063995361328125
loss 0.05395475775003433
loss 0.9179868102073669
loss 0.031049398705363274
loss 0.110157810151577
loss 2.321251630783081
loss 2.638101100921631
loss 0.07739036530256271
loss 1.1791454553604126
loss 3.4927971363067627
loss 0.04094657301902771
loss 0.08401738107204437
loss 0.3709050118923187
loss 1.1301730871200562
loss 0.051736488938331604
loss 0.17805524170398712
loss 2.089721202850342
loss 2.8460960388183594
loss 0.1750801056623459
loss 0.053423985838890076
loss 0.0665794387459755
loss 0.11957761645317078
loss 2.4651644229888916
loss 3.7543435096740723
loss 0.9729767441749573
loss 2.373568296432495
loss 0.025021901354193687
loss 0.03498333692550659
loss 0.7770869731903076
loss 4.80717658996582
loss 0.2395903319120407
loss 1.6837916374206543
loss 0.03980302810668945
loss 1.0685105323791504
loss 3.3574559688568115
loss

loss 2.468144416809082
loss 2.764967441558838
loss 3.3225390911102295
loss 3.055936336517334
loss 0.04543095454573631
loss 2.347073554992676
loss 0.07412351667881012
loss 0.7167453765869141
loss 2.1282405853271484
loss 6.756503582000732
loss 3.401024580001831
loss 0.03549360856413841
loss 0.10953351855278015
loss 0.16277256608009338
loss 0.5167641639709473
loss 0.080140121281147
loss 0.21309693157672882
loss 1.6082043647766113
loss 1.1132640838623047
loss 1.8894336223602295
loss 1.3721576929092407
loss 0.03661416471004486
loss 0.1471681147813797
loss 1.6994954347610474
loss 0.06719601154327393
loss 0.1363423466682434
loss 4.503664970397949
loss 0.21521742641925812
loss 0.05372488126158714
loss 1.5052618980407715
loss 0.03929346054792404
loss 1.5579376220703125
loss 1.3969433307647705
loss 0.018999962136149406
loss 0.7177522778511047
loss 1.6648781299591064
loss 3.3643746376037598
loss 0.11835094541311264
loss 0.022079482674598694
loss 2.9221253395080566
loss 0.06271491199731827
loss 0.

loss 0.5906569957733154
loss 1.1613922119140625
loss 1.8227180242538452
loss 3.737421989440918
loss 1.1964502334594727
loss 0.12564340233802795
loss 1.6012065410614014
loss 0.05050355941057205
loss 1.8384933471679688
loss 0.5882448554039001
loss 0.013351320289075375
loss 0.9039822220802307
loss 1.1868782043457031
loss 0.09563104063272476
loss 2.5427451133728027
loss 6.3360981941223145
loss 0.12896893918514252
loss 0.28944820165634155
loss 0.14567294716835022
loss 0.7000734210014343
loss 0.8014622926712036
loss 0.053764309734106064
loss 0.19062915444374084
loss 0.18827266991138458
loss 0.22097885608673096
loss 0.29096901416778564
loss 3.1958272457122803
loss 0.1279912292957306
loss 0.011940786615014076
loss 0.15738439559936523
loss 1.6108494997024536
loss 0.046942826360464096
loss 2.855647563934326
loss 0.012613048776984215
loss 1.7880810499191284
loss 2.898632049560547
loss 0.03423379734158516
loss 5.524775981903076
loss 0.017790740355849266
loss 0.18829971551895142
loss 0.342415839433

In [216]:
#LSTM testing
test_len =len(test)
with torch.no_grad():
    correct = 0
    for label,sent in test:
        ids=[]
        unk = 3375
        labels=[]
        for word in sent:
            index =indexed_vocab.get(word)
            if index:
                ids.append([indexed_vocab[word]])
            else:
                ids.append([unk])
        sentence=torch.tensor(ids).squeeze()
        batch_outputs = ModelLSTM(sentence)
        predicted = np.argmax(F.softmax(batch_outputs).data.numpy())
        if predicted==label:
             correct+=1
        print('prediction: {}'.format(predicted))
        print('actual: {}'.format(label))
    accuracy = correct/test_len
    print('accuracy: {}'.format(accuracy))

C:\Users\joshy\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


prediction: 17
actual: 17
prediction: 13
actual: 1
prediction: 17
actual: 5
prediction: 24
actual: 24
prediction: 48
actual: 12
prediction: 13
actual: 13
prediction: 1
actual: 12
prediction: 1
actual: 1
prediction: 11
actual: 11
prediction: 17
actual: 1
prediction: 4
actual: 4
prediction: 13
actual: 42
prediction: 24
actual: 24
prediction: 17
actual: 44
prediction: 13
actual: 13
prediction: 17
actual: 17
prediction: 37
actual: 37
prediction: 17
actual: 17
prediction: 2
actual: 2
prediction: 32
actual: 32
prediction: 13
actual: 13
prediction: 4
actual: 4
prediction: 24
actual: 24
prediction: 20
actual: 20
prediction: 20
actual: 20
prediction: 13
actual: 1
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 24
actual: 24
prediction: 10
actual: 10
prediction: 13
actual: 1
prediction: 13
actual: 48
prediction: 11
actual: 11
prediction: 17
actual: 17
prediction: 10
actual: 10
prediction: 45
actual: 45
prediction: 13
actual: 13


prediction: 32
actual: 32
prediction: 37
actual: 37
prediction: 2
actual: 2
prediction: 11
actual: 11
prediction: 11
actual: 11
prediction: 17
actual: 37
prediction: 17
actual: 17
prediction: 17
actual: 42
prediction: 17
actual: 17
prediction: 10
actual: 10
prediction: 32
actual: 32
prediction: 20
actual: 20
prediction: 13
actual: 1
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 39
actual: 39
prediction: 39
actual: 39
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 11
actual: 11
prediction: 13
actual: 12
prediction: 2
actual: 2
prediction: 17
actual: 17
prediction: 17
actual: 41
prediction: 17
actual: 49
prediction: 12
actual: 12
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 48
actual: 50
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 46
actual: 46
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 13
actual: 37
prediction: 39
actual: 39
prediction: 12
actual: 12
prediction: 12
actual: 12
prediction: 13
ac

prediction: 13
actual: 50
prediction: 17
actual: 49
prediction: 17
actual: 17
prediction: 30
actual: 48
prediction: 1
actual: 48
prediction: 17
actual: 37
prediction: 1
actual: 46
prediction: 17
actual: 12
prediction: 17
actual: 17
prediction: 13
actual: 43
prediction: 12
actual: 12
prediction: 17
actual: 48
prediction: 17
actual: 17
prediction: 13
actual: 13
prediction: 13
actual: 5
prediction: 17
actual: 17
prediction: 12
actual: 12
prediction: 11
actual: 11
prediction: 17
actual: 17
prediction: 12
actual: 12
prediction: 13
actual: 50
prediction: 11
actual: 11
prediction: 4
actual: 4
prediction: 17
actual: 48
prediction: 11
actual: 11
prediction: 12
actual: 12
prediction: 17
actual: 48
prediction: 42
actual: 42
prediction: 42
actual: 46
prediction: 32
actual: 32
prediction: 13
actual: 13
prediction: 17
actual: 17
prediction: 11
actual: 18
prediction: 12
actual: 17
prediction: 37
actual: 34
prediction: 12
actual: 12
prediction: 17
actual: 17
prediction: 13
actual: 13
prediction: 17
ac

prediction: 13
actual: 13
prediction: 17
actual: 17
prediction: 42
actual: 17
prediction: 13
actual: 13
prediction: 17
actual: 1
prediction: 13
actual: 48
prediction: 17
actual: 17
prediction: 37
actual: 42
prediction: 17
actual: 42
prediction: 39
actual: 39
prediction: 42
actual: 42
prediction: 17
actual: 17
prediction: 48
actual: 48
prediction: 17
actual: 50
prediction: 4
actual: 43
prediction: 45
actual: 36
prediction: 42
actual: 42
prediction: 12
actual: 12
prediction: 11
actual: 11
prediction: 13
actual: 13
prediction: 12
actual: 12
prediction: 32
actual: 32
prediction: 11
actual: 11
prediction: 13
actual: 13
prediction: 10
actual: 10
prediction: 17
actual: 42
prediction: 2
actual: 2
prediction: 46
actual: 46
prediction: 13
actual: 13
prediction: 45
actual: 45
prediction: 17
actual: 17
prediction: 20
actual: 20
prediction: 10
actual: 10
prediction: 32
actual: 32
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
actual: 17
prediction: 17
a